# Avaliação de Fairness no modelo

Para a avaliação de fairness, será utilizada a métrica Equal opportunity, e será realizada a análise na classe "sexo"

Primeiramente, vamos calcular a métrica de fairness para a classe sexo, analisar, e utilizar técnicas de mitigação de viés a fim de comparar os resultados com o modelo anterior

In [58]:
# Carregando os dados de teste
import pandas as pd
test_data = pd.read_csv('./source/datasets/df_teste.csv')
train_data = pd.read_csv('./source/datasets/df_treino.csv')
print(test_data.columns)
print(test_data.shape)


Index(['Quantidade ovos encontrados', 'Escolaridade', 'Sexo', 'Resultado',
       'Exame_Nao_Realizado', 'Exame_Negativo', 'Exame_Positivo',
       'Tratamento_Nao', 'Tratamento_Sim_Oxaminiquine',
       'Tratamento_Sim_Praziquantel', 'Forma_Aguda', 'Forma_Hepato_Esplenica',
       'Forma_Hepato_Intestinal', 'Forma_Intestinal', 'Forma_Outra', 'Idade',
       'Regiao_Centro-Oeste', 'Regiao_Nordeste', 'Regiao_Norte',
       'Regiao_Sudeste', 'Regiao_Sul'],
      dtype='object')
(1441, 21)


In [59]:
# Removendo colunas que não agregam na avaliação (Assim como realizado no treinamento)
# test_data = test_data.drop(columns=['Exame_Desconhecido', 'Tratamento_Ignorado', 'Forma_Nao_especificado', 'Regiao_Nao_especificado'])

In [82]:
print(test_data['Escolaridade'].value_counts())
print(train_data['Escolaridade'].value_counts())

print("Alfabetizados Teste: ", test_data[test_data['Escolaridade'] == 1]['Resultado'].value_counts())
print("Analfabetos Teste: ", test_data[test_data['Escolaridade'] == 0]['Resultado'].value_counts())

print("Alfabetizados Treino: ", train_data[train_data['Escolaridade'] == 1]['Resultado'].value_counts())
print("Analfabetos treino: ", train_data[train_data['Escolaridade'] == 0]['Resultado'].value_counts())

0    740
1    701
Name: Escolaridade, dtype: int64
1    3162
0    2622
Name: Escolaridade, dtype: int64
Alfabetizados Teste:  0    695
1      6
Name: Resultado, dtype: int64
Analfabetos Teste:  0    714
1     26
Name: Resultado, dtype: int64
Alfabetizados Treino:  0    3124
1      38
Name: Resultado, dtype: int64
Analfabetos treino:  0    2502
1     120
Name: Resultado, dtype: int64


Mortes absolutas por sexo

In [84]:
import pandas as pd
df_train = train_data
df_test = test_data
# Count male deaths (Resultado==1) in the training set (Sexo==1)
male_deaths_train = df_train[df_train['Sexo'] == 1]['Resultado'].value_counts().get(1, 0)
# Count female deaths (Resultado==1) in the training set (Sexo==0)
female_deaths_train = df_train[df_train['Sexo'] == 0]['Resultado'].value_counts().get(1, 0)

# Count male deaths (Resultado==1) in the test set (Sexo==1)
male_deaths_test = df_test[df_test['Sexo'] == 1]['Resultado'].value_counts().get(1, 0)
# Count female deaths (Resultado==1) in the test set (Sexo==0)
female_deaths_test = df_test[df_test['Sexo'] == 0]['Resultado'].value_counts().get(1, 0)

# Create a summary DataFrame with rows for train and test and columns for each sex
data = {
    "Masculino": [male_deaths_train, male_deaths_test],
    "Feminino": [female_deaths_train, female_deaths_test]
}
df_summary = pd.DataFrame(data, index=["Treino", "Teste"])

print(df_summary)

        Masculino  Feminino
Treino         81        77
Teste          17        15


In [88]:
import pandas as pd
df_train = train_data
df_test = test_data
# Count male deaths (Resultado==1) in the training set (Sexo==1)
male_deaths_train = df_train[df_train['Escolaridade'] == 1]['Resultado'].value_counts().get(1, 0)
# Count female deaths (Resultado==1) in the training set (Escolaridade==0)
female_deaths_train = df_train[df_train['Escolaridade'] == 0]['Resultado'].value_counts().get(1, 0)

# Count male deaths (Resultado==1) in the test set (Escolaridade==1)
male_deaths_test = df_test[df_test['Escolaridade'] == 1]['Resultado'].value_counts().get(1, 0)
# Count female deaths (Resultado==1) in the test set (Escolaridade==0)
female_deaths_test = df_test[df_test['Escolaridade'] == 0]['Resultado'].value_counts().get(1, 0)

# Create a summary DataFrame with rows for train and test and columns for each sex
data = {
    "Alfabetizado": [male_deaths_train, male_deaths_test],
    "Não-Alfabetizado": [female_deaths_train, female_deaths_test]
}
df_summary = pd.DataFrame(data, index=["Treino", "Teste"])

print(df_summary)

        Alfabetizado  Não-Alfabetizado
Treino            38               120
Teste              6                26


Taxa de mortalidade por sexo em cada conjunto

In [92]:
import pandas as pd

# Calculate death rates for the training data (as fractions) using normalize=True:
male_death_rate_train = df_train[df_train['Sexo'] == 1]['Resultado'].mean()
female_death_rate_train = df_train[df_train['Sexo'] == 0]['Resultado'].mean()

# Calculate death rates for the test data (as fractions)
male_death_rate_test = df_test[df_test['Sexo'] == 1]['Resultado'].mean()
female_death_rate_test = df_test[df_test['Sexo'] == 0]['Resultado'].mean()

# Convert the rates to readable percentages with two decimals and a comma as decimal separator
male_death_rate_train_str = f"{male_death_rate_train * 100:.2f}%".replace('.', ',')
female_death_rate_train_str = f"{female_death_rate_train * 100:.2f}%".replace('.', ',')
male_death_rate_test_str = f"{male_death_rate_test * 100:.2f}%".replace('.', ',')
female_death_rate_test_str = f"{female_death_rate_test * 100:.2f}%".replace('.', ',')

# Create a summary DataFrame with the percentages as strings
data = {
    "Feminino": [male_death_rate_train_str, male_death_rate_test_str],
    "Masculino": [female_death_rate_train_str, female_death_rate_test_str]
}

df_summary_rates = pd.DataFrame(data, index=["Treino", "Teste"])
df_summary_rates

,Feminino,Masculino
Treino,"3,52%","2,21%"
Teste,"2,97%","1,73%"


In [90]:
import pandas as pd

# Calculate death rates for the training data (as fractions) using normalize=True:
alfa_death_rate_train = df_train[df_train['Escolaridade'] == 1]['Resultado'].value_counts(normalize=True).get(1, 0)
nao_alfa__death_rate_train = df_train[df_train['Escolaridade'] == 0]['Resultado'].value_counts(normalize=True).get(1, 0)

# Calculate death rates for the test data (as fractions)
alfa_death_rate_test = df_test[df_test['Escolaridade'] == 1]['Resultado'].value_counts(normalize=True).get(1, 0)
nao_alfa__death_rate_test = df_test[df_test['Escolaridade'] == 0]['Resultado'].value_counts(normalize=True).get(1, 0)

# Convert the rates to readable percentages with two decimals and a comma as decimal separator
alfa_death_rate_train_str = f"{alfa_death_rate_train * 100:.2f}%".replace('.', ',')
nao_alfa_death_rate_train_str = f"{nao_alfa__death_rate_train * 100:.2f}%".replace('.', ',')
alfa_death_rate_test_str = f"{alfa_death_rate_test * 100:.2f}%".replace('.', ',')
nao_alfa_death_rate_test_str = f"{nao_alfa__death_rate_test * 100:.2f}%".replace('.', ',')

# Create a summary DataFrame with the percentages as strings
data = {
    "Alfabetizado": [alfa_death_rate_train_str, alfa_death_rate_test_str],
    "Não Alfabetizado": [nao_alfa_death_rate_train_str, nao_alfa_death_rate_test_str]
}

df_summary_rates = pd.DataFrame(data, index=["Treino", "Teste"])
df_summary_rates

,Alfabetizado,Não Alfabetizado
Treino,"1,20%","4,58%"
Teste,"0,86%","3,51%"


In [62]:
# Dividindo os conjuntos de predições e target
X_test = test_data.drop(columns=['Resultado'])
y_test = test_data['Resultado']
sexo = test_data['Sexo']
escolaridade = test_data['Escolaridade']

In [63]:
# Carregando o modelo
import pickle
with open('source/models/best_model.pkl', 'rb') as file:
    best_model = pickle.load(file)


Realizando as predições no conjunto de teste para o melhor modelo encontrado: 

In [64]:
X_test.columns

Index(['Quantidade ovos encontrados', 'Escolaridade', 'Sexo',
       'Exame_Nao_Realizado', 'Exame_Negativo', 'Exame_Positivo',
       'Tratamento_Nao', 'Tratamento_Sim_Oxaminiquine',
       'Tratamento_Sim_Praziquantel', 'Forma_Aguda', 'Forma_Hepato_Esplenica',
       'Forma_Hepato_Intestinal', 'Forma_Intestinal', 'Forma_Outra', 'Idade',
       'Regiao_Centro-Oeste', 'Regiao_Nordeste', 'Regiao_Norte',
       'Regiao_Sudeste', 'Regiao_Sul'],
      dtype='object')

### Avaliando para sexo

In [65]:
y_pred = best_model.predict(X_test)

print(sexo.unique())

[1 0]


/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


No nosso caso, o valor 0 é masculino e 0 é feminino

In [66]:
from sklearn.metrics import recall_score

grupos = sexo.unique()
labels = ['Masculino', 'Feminino']
for value in grupos:
    idx_gp = sexo == value # Escolhe o sexo definido e calcula o recall
    recall = recall_score(y_test[idx_gp], y_pred[idx_gp])
    print(f'Recall para o grupo {labels[value]}: {recall:.4f}')

Recall para o grupo Feminino: 0.6471
Recall para o grupo Masculino: 0.8667


### Avaliando para escolaridade

Testando para escolaridade

In [67]:
escolaridade_labels = {
    0: "Analfabeto",
    1: "1ª a 4ª série incompleta",
    2: "4ª série completa",
    3: "5ª a 8ª série incompleta",
    4: "Ensino fundamental completo",
    5: "Ensino médio incompleto",
    6: "Ensino médio completo",
    7: "Educação superior incompleta",
    8: "Educação superior completa",
}

for value in escolaridade_labels.keys():
    idx_gp = escolaridade == value # Escolhe a escolaridade definida e calcula o recall
    recall = recall_score(y_test[idx_gp], y_pred[idx_gp])
    print(f'Recall para o grupo {escolaridade_labels[value]}: {recall:.4f}')

Recall para o grupo Analfabeto: 0.7000
Recall para o grupo 1ª a 4ª série incompleta: 1.0000
Recall para o grupo 4ª série completa: 1.0000
Recall para o grupo 5ª a 8ª série incompleta: 0.0000
Recall para o grupo Ensino fundamental completo: 0.5000
Recall para o grupo Ensino médio incompleto: 1.0000
Recall para o grupo Ensino médio completo: 0.0000
Recall para o grupo Educação superior incompleta: 0.0000
Recall para o grupo Educação superior completa: 0.0000


/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Agora, vamos avaliar a métrica em relação a uma avaliação binária da escolaridade

In [68]:
def nova_Escolaridade(x):
    if x < 2:
        return 0
    else: 
        return 1


In [69]:
escolaridade_bin = escolaridade.apply(lambda x: nova_Escolaridade(x))  # Agrupando em ensino superior completo ou não

In [70]:
print(escolaridade.value_counts())
print(escolaridade_bin.value_counts())

0    619
6    212
3    135
1    121
2    104
5    101
4     83
8     48
7     18
Name: Escolaridade, dtype: int64
0    740
1    701
Name: Escolaridade, dtype: int64


In [71]:
escolaridade_bin_labels = {
    0: "Não alfabetizado",
    1: "Alfabetizado",
}

for value in escolaridade_bin_labels.keys():
    idx_gp = escolaridade_bin == value # Escolhe a escolaridade definida e calcula o recall
    recall = recall_score(y_test[idx_gp], y_pred[idx_gp])
    print(f'Recall para o grupo {escolaridade_bin_labels[value]}: {recall:.4f}')
    print(f"Mortes para o grupo {escolaridade_bin_labels[value]}: {test_data[y_test == 1][idx_gp].shape[0]}")
    print("---------------------------------------------------------------")

Recall para o grupo Não alfabetizado: 0.7692
Mortes para o grupo Não alfabetizado: 26
---------------------------------------------------------------
Recall para o grupo Alfabetizado: 0.6667
Mortes para o grupo Alfabetizado: 6
---------------------------------------------------------------


/tmp/ipykernel_16801/995386746.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(f"Mortes para o grupo {escolaridade_bin_labels[value]}: {test_data[y_test == 1][idx_gp].shape[0]}")
/tmp/ipykernel_16801/995386746.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(f"Mortes para o grupo {escolaridade_bin_labels[value]}: {test_data[y_test == 1][idx_gp].shape[0]}")


In [72]:
import pandas as pd
from sklearn.metrics import recall_score

# Initialize an empty dictionary to store the recall for each escolaridade group
recall_dict = {}

for value, label in escolaridade_bin_labels.items():
    # Create a boolean mask for the given escolaridade value 
    idx_gp = escolaridade_bin == value
    # Calculate recall for that group using the values from y_test and y_pred
    recall_val = recall_score(y_test[idx_gp], y_pred[idx_gp])
    # Save the recall in the dictionary with the descriptive label as key
    recall_dict[label] = recall_val
    
# Transform the recall_dict into a DataFrame with a single row
df_recall = pd.DataFrame([recall_dict])

df_recall

,Não alfabetizado,Alfabetizado
0,0.769231,0.666667


Talvez chegue no final e perceba que nao faz sentido usar essa métrica de fairness, porque estamos tentando ajustar baseado no que já tínhamos antes (não entendi muito bem)

In [73]:
train_data = pd.read_csv('./source/datasets/df_treino.csv')
test_data = pd.read_csv('./source/datasets/df_teste.csv')

# Ajustado os conjuntos para preduções binárias da escolaridade
train_data['Escolaridade'] = train_data['Escolaridade'].apply(lambda x: nova_Escolaridade(x))
test_data['Escolaridade'] = test_data['Escolaridade'].apply(lambda x: nova_Escolaridade(x))

# Salvando os conjuntos ajustados
train_data.to_csv('entrega-1/df_treino_fairness.csv', index=False)
test_data.to_csv('entrega-1/df_teste_fairness.csv', index=False)

## Comparando o resultado com o modelo treinado balanceando os dados da classe sexo

In [95]:
# Carregando o modelo
import pickle
with open('source/models/best_model_fairness_sexo.pkl', 'rb') as file:
    best_model_fairness = pickle.load(file)


In [96]:
from sklearn.metrics import recall_score
y_pred_f = best_model_fairness.predict(X_test)

sexo_bin_labels = {
    0: "Masculino",
    1: "Feminino",
}

for value in sexo_bin_labels.keys():
    idx_gp = sexo == value # Escolhe a sexo definida e calcula o recall
    recall = recall_score(y_test[idx_gp], y_pred_f[idx_gp])
    print(f'Recall para o grupo {sexo_bin_labels[value]}: {recall:.4f}')
    # print(f"Mortes para o grupo {sexo_bin_labels[value]}: {test_data[y_test == 1][idx_gp].shape[0]}")
    # print("---------------------------------------------------------------")

Recall para o grupo Masculino: 0.7333
Recall para o grupo Feminino: 0.5882


/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
